In [10]:
import logging
from pathlib import Path
from data_loader import BasicDataset, RemoteSensingDataset

import torch
from torch.utils.data import DataLoader, random_split
from torch_utils import select_device

# device = "cuda" if torch.cuda.is_availble() else "cpu"
device_option = 'cuda:0'
device = select_device(device='cpu' if torch.cuda.is_available() is False else device_option)
logging.info(f'You are using {device}')

dir_img = Path(r'./data_for_training2/images/')
dir_mask = Path(r'./data_for_training2/labels/')
img_scale : float = 0.50


# 1. Create dataset
try:
    dataset = RemoteSensingDataset(dir_img, dir_mask, img_scale)
except (AssertionError, RuntimeError):
    dataset = BasicDataset(dir_img, dir_mask, img_scale)

[WARNING] You are using CPU - it might slow and break your program



In [12]:
import torch.cuda.profiler as profiler 
# import pyprof 
# pyprof.init()

In [11]:
print(torch.__version__)

1.4.0


In [ ]:
# 2. Split into train / validation partitions
n_val = int(len(dataset) * 0.15)
n_train = len(dataset) - n_val

print(n_val, n_train)

In [ ]:
train_set, val_set = random_split(dataset,[n_train, n_val])

print(train_set, val_set)

In [ ]:
# 3. Create data loaders
loader_args = dict(batch_size=32, num_workers=8, pin_memory=True)
train_loader = DataLoader(train_set, shuffle=True, **loader_args)
val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args)


In [ ]:
# import wandb

# # (Initialize logging)
# experiment = wandb.init(project='U-Net', resume='allow', anonymous='must')
# experiment.config.update(dict(epochs=epochs, batch_size=batch_size, learning_rate=learning_rate,
#                               val_percent=val_percent, save_checkpoint=save_checkpoint, img_scale=img_scale,
#                                   amp=amp))

# logging.info(f'''Starting training:
#              Epochs:          {epochs}
#              Batch size:      {batch_size}
#              Learning rate:   {learning_rate}
#              Training size:   {n_train}
#              Validation size: {n_val}
#              Checkpoints:     {save_checkpoint}
#              Device:          {device.type}
#              Images scaling:  {img_scale}
#              Mixed Precision: {amp}
#              ''')

In [ ]:
# 4. Set up the optimizer, the loss, the learning rate scheduler and the loss scaling for AMP
optimizer = optim.RMSprop(net.parameters(), lr=learning_rate, weight_decay=1e-8, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  # goal: maximize Dice score
grad_scaler = torch.cuda.amp.GradScaler(enabled=amp)
criterion = nn.CrossEntropyLoss()
global_step = 0